# MOUNT

In [79]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Initial Setup

In [70]:
# Cython needs to be installed before pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib

In [69]:
# Downgrade torch to accommodate DCNv2
!pip install torchvision==0.5.0
!pip install torch==1.4.0


     |████████████████████████████████| 4.0MB 14.1MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


# Clone YolAct

In [5]:
# Make sure we're in the top folder
# %cd /content

# # Clone the repo
# !git clone https://github.com/dbolya/yolact.git

/content
Cloning into 'yolact'...
remote: Enumerating objects: 2936, done.
remote: Total 2936 (delta 0), reused 0 (delta 0), pack-reused 2936
Receiving objects: 100% (2936/2936), 21.21 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (1998/1998), done.


In [3]:
!ls /content

gdrive	sample_data


In [36]:
# !tar -xzvf '/content/gdrive/MyDrive/Colab_Notebooks/darknet/cudnn/cudnn-10.1-linux-x64-v8.0.5.39.tgz' -C /usr/local/

cuda/include/cudnn.h
cuda/include/cudnn_adv_infer.h
cuda/include/cudnn_adv_train.h
cuda/include/cudnn_backend.h
cuda/include/cudnn_cnn_infer.h
cuda/include/cudnn_cnn_train.h
cuda/include/cudnn_ops_infer.h
cuda/include/cudnn_ops_train.h
cuda/include/cudnn_version.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.8
cuda/lib64/libcudnn.so.8.0.5
cuda/lib64/libcudnn_adv_infer.so
cuda/lib64/libcudnn_adv_infer.so.8
cuda/lib64/libcudnn_adv_infer.so.8.0.5
cuda/lib64/libcudnn_adv_train.so
cuda/lib64/libcudnn_adv_train.so.8
cuda/lib64/libcudnn_adv_train.so.8.0.5
cuda/lib64/libcudnn_cnn_infer.so
cuda/lib64/libcudnn_cnn_infer.so.8
cuda/lib64/libcudnn_cnn_infer.so.8.0.5
cuda/lib64/libcudnn_cnn_train.so
cuda/lib64/libcudnn_cnn_train.so.8
cuda/lib64/libcudnn_cnn_train.so.8.0.5
cuda/lib64/libcudnn_ops_infer.so
cuda/lib64/libcudnn_ops_infer.so.8
cuda/lib64/libcudnn_ops_infer.so.8.0.5
cuda/lib64/libcudnn_ops_train.so
cuda/lib64/libcudnn_ops_train.so.8
cuda/lib64/libcudnn_o

In [41]:
!ls /usr/local/cuda/include | grep cuda

cuda_device_runtime_api.h
cudaEGL.h
cuda_egl_interop.h
cuda_fp16.h
cuda_fp16.hpp
cudaGL.h
cuda_gl_interop.h
cuda.h
cudalibxt.h
cuda_occupancy.h
cuda_profiler_api.h
cudaProfiler.h
cudart_platform.h
cuda_runtime_api.h
cuda_runtime.h
cuda_surface_types.h
cuda_texture_types.h
cudaVDPAU.h
cuda_vdpau_interop.h


In [60]:
cuda_v = !/usr/local/cuda/bin/nvcc --version
gcc_v = !gcc --version
g_v = !g++ --version

In [61]:
print('cuda_version : {}'.format(cuda_v[3]),'\n','gcc_version : {}'.format(gcc_v[0]),'\n','g++_version : {}'.format(g_v[0]))
#Cuda 10.1 버전의 max gcc version은 8이라 노 프라블럼
# 버전 호환 검색 결과 :  https://www.google.com/search?rlz=1C5CHFA_enKR789KR789&sxsrf=ALeKk02Qaxql46zVnCYkJ4O4klUXwIj2gA%3A1612074330193&ei=Wk0WYNiqC5rZ-Qbrh6XwBQ&q=cuda+10.1+gcc+version&oq=cuda+10.1+gcc+version&gs_lcp=CgZwc3ktYWIQAzICCAA6CggAELADEBQQhwI6CAgAELADEMsBOgUIABCwAzoHCAAQsAMQHjoJCAAQsAMQCBAeOgkIABCwAxAHEB46BAgjECc6BggAEAcQHjoICAAQCBAHEB46BAgAEB5QlkhYjWhg_W5oAnAAeACAAXuIAcAFkgEDMC42mAEAoAEBqgEHZ3dzLXdpesgBCsABAQ&sclient=psy-ab&ved=0ahUKEwjYv7XuxMXuAhWabN4KHetDCV4Q4dUDCA0&uact=5

cuda_version : Cuda compilation tools, release 10.1, V10.1.243 
 gcc_version : gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0 
 g++_version : g++ (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0


In [68]:
!cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

#define CUDNN_MAJOR 8
#define CUDNN_MINOR 0
#define CUDNN_PATCHLEVEL 5
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#endif /* CUDNN_VERSION_H */


# DCNv2

In [71]:
# Change to the right directory
%cd /content/gdrive/MyDrive/Colab_Notebooks/yolo-act/yolact/external/DCNv2

# Build DCNv2
!python setup.py build develop

/content/gdrive/MyDrive/Colab_Notebooks/yolo-act/yolact/external/DCNv2
running build
running build_ext
building '_ext' extension
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/content/gdrive/MyDrive/Colab_Notebooks/yolo-act/yolact/external/DCNv2/src -I/usr/local/lib/python3.6/dist-packages/torch/include -I/usr/local/lib/python3.6/dist-packages/torch/include/torch/csrc/api/include -I/usr/local/lib/python3.6/dist-packages/torch/include/TH -I/usr/local/lib/python3.6/dist-packages/torch/include/THC -I/usr/local/cuda/include -I/usr/include/python3.6m -c /content/gdrive/MyDrive/Colab_Notebooks/yolo-act/yolact/external/DCNv2/src/vision.cpp -o build/temp.linux-x86_64-3.6/content/gdrive/MyDrive/Colab_Notebooks/yolo-act/yolact/external/DCNv2/src/vision.o -DTORCH_API_INCLUDE_EXTENSION_H -DTORCH_EXTENSION_NAME=_ext -D_GLIBCXX_USE_CXX11_ABI=0 -std=c++11
x86_64-linux-gnu-gcc

# Pretrained Weights

In [6]:
# Make sure we're in the top folder
%cd /content

# Clone the repo
!git clone https://github.com/chentinghao/download_google_drive.git

# Create a new directory for the pre-trained weights
!mkdir -p /content/yolact/weights

# Download the file
!python ./download_google_drive/download_gdrive.py 1ZPu1YR2UzGHQD0o1rEqy-j5bmEm3lbyP ./yolact/weights/yolact_plus_resnet50_54_800000.pth

/content
Cloning into 'download_google_drive'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
129MB [00:01, 104MB/s] 


# Get test images

In [ ]:
# Make a new directory for the test images
!mkdir /content/test_images

# Move to the new directory
%cd /content/test_images

# Make a list of URLs from the COCO dataset
get_imgs = ['http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg',
            'http://farm3.staticflickr.com/2835/9520716824_51220b13d0_z.jpg',
            'http://farm3.staticflickr.com/2817/10105052924_0c8d79e62c_z.jpg',
            'https://farm3.staticflickr.com/2637/3679968895_37a79e55b3_z.jpg',
            'https://farm8.staticflickr.com/7137/7705349298_6bea29cf1c_z.jpg',
            'http://farm5.staticflickr.com/4105/4981945644_3f162314ed_z.jpg',
            'http://farm1.staticflickr.com/40/101757885_a8fc40e273_z.jpg',
            'http://farm3.staticflickr.com/2654/3897535041_fa8dd5570b_z.jpg']

# Download all the images
for img_url in get_imgs:
  !wget {img_url} -nc

# List the contents of the folder. Should have images.
!pwd
!ls

# Run Inference on images

In [8]:
# Move up to the top level directory
%cd /content

# Delete any previous output_images folder (in case we're running this more than once)
!rm /content/output_images -r

# Create a new directory for output images
!mkdir -p /content/output_images

# Run inference using our pre-trained weights on all images in the directory
!python ./yolact/eval.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth --config=yolact_plus_resnet50_config --score_threshold=0.15 --top_k=15 --images=test_images:output_images

/content
rm: cannot remove '/content/output_images': No such file or directory
Loading model...Traceback (most recent call last):
  File "./yolact/eval.py", line 1097, in <module>
    net = Yolact()
  File "/content/yolact/yolact.py", line 402, in __init__
    self.backbone = construct_backbone(cfg.backbone)
  File "/content/yolact/backbone.py", line 451, in construct_backbone
    backbone = cfg.type(*cfg.args)
  File "/content/yolact/backbone.py", line 83, in __init__
    self._make_layer(block, 128, layers[1], stride=2, dcn_layers=dcn_layers[1], dcn_interval=dcn_interval)
  File "/content/yolact/backbone.py", line 114, in _make_layer
    layers.append(block(self.inplanes, planes, stride, downsample, self.norm_layer, self.dilation, use_dcn=use_dcn))
  File "/content/yolact/backbone.py", line 23, in __init__
    padding=dilation, dilation=dilation, deformable_groups=1)
  File "/content/yolact/backbone.py", line 11, in DCN
    raise Exception('DCN could not be imported. If you want to u

# Display output images

In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

output_images = Path('output_images')

def show_image(img_path):
  img = cv2.imread(img_path)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.show()

# Iterate through all of the output images and display them
for img_path in output_images.iterdir():
  print(img_path)
  show_image(str(img_path))

In [ ]:
## ㅋㅋ
from IPython.display import HTML
HTML('<iframe width="1280" height="720" src="https://www.youtube.com/embed/bQgtmbzN7jg" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [ ]:
# Make a new directory for the test images
# !mkdir -p /content/test_video

# Move to the top level directory
# %cd /content

# Download the file
# https://drive.google.com/file/d/1I4fivRLniVNt_LcWqhQNehTSdEkZ4ytW/view?usp=sharing
# Note that file_id is extracted from the share URL above, it'll be different for every file
# file_id = "1I4fivRLniVNt_LcWqhQNehTSdEkZ4ytW"
file_path = "/content/gdrive/MyDrive/Colab_Notebooks/darknet/bin/darknet/fastandfurious_short.mov" # <- this is the destination path

# !python /content/download_google_drive/download_gdrive.py {file_id} {file_path}

!ls /content/test_video

In [21]:
!ls /content/gdrive/MyDrive/Colab_Notebooks/darknet/bin/darknet

bad.list	       darknet			 predictions.jpg  train
chart.png	       fastandfurious_short.mov  result_past	  train_2
chart_yolov3-tiny.png  frame29.jpg		 test.mov


In [ ]:
%cd /content/gdrive/MyDrive/Colab_Notebooks/darknet/bin/darknet
!mkdir -p /content/gdrive/MyDrive/Colab_Notebooks/darknet/bin/darknet/output_videos
file_path = "/content/gdrive/MyDrive/Colab_Notebooks/darknet/bin/darknet/6-3.mov" # <- this is the destination path
output_path = "/content/gdrive/MyDrive/Colab_Notebooks/darknet/bin/darknet/output_videos/6-3_test.mov"
!python /content/gdrive/MyDrive/Colab_Notebooks/yolo-act/yolact/eval.py --trained_model=/content/gdrive/MyDrive/Colab_Notebooks/yolo-act/yolact/weights/yolact_plus_resnet50_54_800000.pth --score_threshold=0.15 --top_k=15 --video_multiframe=4 --video={file_path}:{output_path}

Output hidden; open in https://colab.research.google.com to view.

In [73]:
!ls /content/gdrive/MyDrive/Colab_Notebooks/yolo-act/

download_google_drive  output_images  results  test_images  yolact


In [35]:
!ls /usr/local/cuda/include | grep cuda

cuda_device_runtime_api.h
cudaEGL.h
cuda_egl_interop.h
cuda_fp16.h
cuda_fp16.hpp
cudaGL.h
cuda_gl_interop.h
cuda.h
cudalibxt.h
cuda_occupancy.h
cuda_profiler_api.h
cudaProfiler.h
cudart_platform.h
cuda_runtime_api.h
cuda_runtime.h
cuda_surface_types.h
cuda_texture_types.h
cudaVDPAU.h
cuda_vdpau_interop.h
